# Генератор наборов данных

## Генератор набора данных "Вторая итерация"


| № п/п | Акроним поля           | Принятое решение                                                   |
|-------|------------------------|--------------------------------------------------------------------|
| 1     | MRG                    | Удалить из набора данных                                           |
| 2     | REGION                 | Заполнить признаком “OTHER”                                        |
| 3     | MONTANT                | Заполнить значением 0                                              |
| 4     | FREQUENCE_RECH         | Заполнить значением 0                                              |
| 5     | REVENUE                | Заполнить значением 0                                              |
| 6     | ARPU_SEGMENT           | Заполнить значением 0                                              |
| 7     | FREQUENCE              | Заполнить значением 0                                              |
| 8     | DATA_VOLUME            | Заполнить значением 0                                              |
| 9     | ON_NET                 | Заполнить значением 0                                              |
| 10    | ORANGE                 | Заполнить значением 0                                              |
| 11    | TIGO                   | Заполнить значением 0                                              |
| 12    | ZONE1                  | Заполнить значением 0                                              |
| 13    | ZONE2                  | Заполнить значением 0                                              |
| 14    | TOP_PACK               | С помощью кластеризации определить возможное значение              |
| 15    | MONTANT/FREQUENCE_RECH | Дополнительный параметр созданный делением соответствующих колонок |
| 16    | REVENUE/FREQUENCE_RECH | Дополнительный параметр созданный делением соответствующих колонок |
| 17    | DATA_VOLUME/REGULARITY | Дополнительный параметр созданный делением соответствующих колонок |
| 18    | ON_NET/REGULARITY      | Дополнительный параметр созданный делением соответствующих колонок |
| 19    | ORANGE/REGULARITY      | Дополнительный параметр созданный делением соответствующих колонок |
| 20    | TIGO/REGULARITY        | Дополнительный параметр созданный делением соответствующих колонок |
| 21    | ZONE1/REGULARITY       | Дополнительный параметр созданный делением соответствующих колонок |
| 22    | ZONE2/REGULARITY       | Дополнительный параметр созданный делением соответствующих колонок |



In [1]:
import pickle

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
from sklearn.metrics import rand_score

In [2]:
test_df = pd.read_csv('Test.csv')
train_df = pd.read_csv('Train.csv')

In [3]:
train_df

,user_id,REGION,TENURE,MONTANT,FREQUENCE_RECH,REVENUE,ARPU_SEGMENT,FREQUENCE,DATA_VOLUME,ON_NET,ORANGE,TIGO,ZONE1,ZONE2,MRG,REGULARITY,TOP_PACK,FREQ_TOP_PACK,CHURN
0,00000bfd7d50f01092811bc0c8d7b0d6fe7c3596,FATICK,K > 24 month,4250.0,15.0,4251.0,1417.0,17.0,4.0,388.0,46.0,1.0,1.0,2.0,NO,54,On net 200F=Unlimited _call24H,8.0,0
1,00000cb4a5d760de88fecb38e2f71b7bec52e834,NaN,I 18-21 month,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NO,4,NaN,NaN,1
2,00001654a9d9f96303d9969d0a4a851714a4bb57,NaN,K > 24 month,3600.0,2.0,1020.0,340.0,2.0,NaN,90.0,46.0,7.0,NaN,NaN,NO,17,On-net 1000F=10MilF;10d,1.0,0
3,00001dd6fa45f7ba044bd5d84937be464ce78ac2,DAKAR,K > 24 month,13500.0,15.0,13502.0,4501.0,18.0,43804.0,41.0,102.0,2.0,NaN,NaN,NO,62,"Data:1000F=5GB,7d",11.0,0
4,000028d9e13a595abe061f9b58f3d76ab907850f,DAKAR,K > 24 month,1000.0,1.0,985.0,328.0,1.0,NaN,39.0,24.0,NaN,NaN,NaN,NO,11,Mixt 250F=Unlimited_call24H,2.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2154043,ffffe85215ddc71a84f95af0afb0deeea90e6967,NaN,K > 24 month,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NO,6,NaN,NaN,0
2154044,ffffeaaa9289cdba0ac000f0ab4b48f4aa74ed15,THIES,K > 24 month,6100.0,15.0,5800.0,1933.0,15.0,621.0,26.0,40.0,40.0,NaN,NaN,NO,55,"Data: 200 F=100MB,24H",9.0,0
2154045,fffff172fda1b4bb38a95385951908bb92379809,NaN,K > 24 month,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NO,1,NaN,NaN,1
2154046,fffff5911296937a37f09a37a549da2e0dad6dbb,THIES,K > 24 month,10000.0,11.0,7120.0,2373.0,13.0,NaN,0.0,140.0,13.0,NaN,NaN,NO,28,All-net 500F=2000F;5d,12.0,0


In [4]:
top_pack_cats = {'unlimited': [], 'data': [], 'mixt': [], 'net': [], 'evc': [], 'daily': [], 'week': [],
                 'month': [], 'pilot': [], 'gprs': [], 'wifi': [], 'internat': [], 'other': []}


def top_pack_coding(top_pack_value):
    global top_pack_cats
    for key in top_pack_cats.keys():
        if str(top_pack_value).lower() in top_pack_cats[key]:
            return key


def convert_packs(df: pd.DataFrame) -> pd.DataFrame:
    global top_pack_cats

    packs = list(df['TOP_PACK'].unique())

    k = tuple(top_pack_cats.keys())

    for pack in packs:
        if str(k[0]) in str(pack).lower():
            top_pack_cats[k[0]].append(str(pack).lower())
        elif str(k[1]) in str(pack).lower():
            top_pack_cats[k[1]].append(str(pack).lower())
        elif str(k[2]) in str(pack).lower():
            top_pack_cats[k[2]].append(str(pack).lower())
        elif str(k[3]) in str(pack).lower():
            top_pack_cats[k[3]].append(str(pack).lower())
        elif str(k[4]) in str(pack).lower():
            top_pack_cats[k[4]].append(str(pack).lower())
        elif str(k[5]) in str(pack).lower():
            top_pack_cats[k[5]].append(str(pack).lower())
        elif str(k[6]) in str(pack).lower():
            top_pack_cats[k[6]].append(str(pack).lower())
        elif str(k[7]) in str(pack).lower():
            top_pack_cats[k[7]].append(str(pack).lower())
        elif str(k[8]) in str(pack).lower():
            top_pack_cats[k[8]].append(str(pack).lower())
        elif str(k[9]) in str(pack).lower():
            top_pack_cats[k[9]].append(str(pack).lower())
        elif str(k[10]) in str(pack).lower():
            top_pack_cats[k[10]].append(str(pack).lower())
        elif str(k[11]) in str(pack).lower():
            top_pack_cats[k[11]].append(str(pack).lower())
        else:
            top_pack_cats[k[12]].append(str(pack).lower())

    print(f'Разнесено названий по категориям: {str([len(top_pack_cats[_]) for _ in top_pack_cats.keys()])}')

    df['TOP_PACK'] = df['TOP_PACK'].apply(top_pack_coding)
    return df

In [10]:
def convert_df_to_it2(it1_df: pd.DataFrame, is_train_df: bool) -> pd.DataFrame:
    it1_df = it1_df.drop(columns=['MRG'])
    # заполняем и переводим в int для оптимизации памяти
    it1_df["REGION"] = it1_df["REGION"].fillna("OTHER")
    it1_df["MONTANT"] = it1_df["MONTANT"].fillna(0)
    it1_df["FREQUENCE_RECH"] = it1_df["FREQUENCE_RECH"].fillna(0)
    it1_df["REVENUE"] = it1_df["REVENUE"].fillna(0)
    it1_df["ARPU_SEGMENT"] = it1_df["ARPU_SEGMENT"].fillna(0)
    it1_df["FREQUENCE"] = it1_df["FREQUENCE"].fillna(0)
    it1_df["DATA_VOLUME"] = it1_df["DATA_VOLUME"].fillna(0)
    it1_df["ON_NET"] = it1_df["ON_NET"].fillna(0)
    it1_df["ORANGE"] = it1_df["ORANGE"].fillna(0)
    it1_df["TIGO"] = it1_df["TIGO"].fillna(0)
    it1_df["ZONE1"] = it1_df["ZONE1"].fillna(0)
    it1_df["ZONE2"] = it1_df["ZONE2"].fillna(0)

    columns_to_drop = ['TOP_PACK', 'FREQ_TOP_PACK', 'user_id']
    cluster_model_file = 'TOP_PACK_predict_model_test.pkl'

    if is_train_df:
        columns_to_drop.append('CHURN')
        cluster_model_file = 'TOP_PACK_predict_model.pkl'

    ten_reg_df = it1_df[['user_id', "TENURE", "REGION"]]
    it1_df = pd.get_dummies(it1_df, columns=['TENURE', 'REGION'])

    top_pack_predict_df = it1_df[it1_df["TOP_PACK"].isna()].drop(
        columns=columns_to_drop)

    # набор данных для обучения и теста метода KNN для дальнейшего предсказания TOP_PACK
    # разделение на параметры и целевую переменную
    top_pack_traintest_df = it1_df[~it1_df["TOP_PACK"].isna()]
    top_pack_X_df = top_pack_traintest_df.drop(columns=columns_to_drop)
    top_pack_y_df = top_pack_traintest_df['TOP_PACK']

    # создаем модель для предсказания TOP_PACK методом кластеризации,
    # количество кластеров == кол-ву возможных значений TOP_PACK
    cluster_model1 = KMeans(n_clusters=len(it1_df['TOP_PACK'].unique()))
    top_pack_X_df_train, top_pack_X_df_test, top_pack_y_df_train, top_pack_y_df_test = train_test_split(top_pack_X_df,
                                                                                                        top_pack_y_df,
                                                                                                        train_size=0.7,
                                                                                                        random_state=777)
    # проверяем если модельку уже делали, то просто загружаем её, если нет - обучаем
    try:
        cluster_model1 = pickle.load(open(cluster_model_file, 'rb'))
    except FileNotFoundError:
        print('File not found, training new model')
        cluster_model1.fit(top_pack_X_df_train)
        pickle.dump(cluster_model1, open(cluster_model_file, 'wb'))

    predicted_y_train = cluster_model1.predict(top_pack_X_df_train)
    predicted_y_test = cluster_model1.predict(top_pack_X_df_test)

    print(
        f"Результаты кластеризации:\nТренировочная выборка: {rand_score(top_pack_y_df_train, predicted_y_train)}\nТестовая выборка: {rand_score(top_pack_y_df_test, predicted_y_test)}")

    # заполняем отсутствующие значения "TOP_PACK"
    top_pack_predict_df_values = cluster_model1.predict(top_pack_predict_df)
    top_pack_labels = it1_df['TOP_PACK'].unique()
    top_pack_predict_df['TOP_PACK'] = top_pack_labels[top_pack_predict_df_values]
    top_pack_y_df = top_pack_y_df.append(top_pack_predict_df['TOP_PACK']).sort_index()
    it1_df['TOP_PACK'] = top_pack_y_df
    it1_df['REGION'] = ten_reg_df['REGION']
    it1_df['TENURE'] = ten_reg_df['TENURE']

    it1_df = it1_df.drop(
        columns=['TENURE_D 3-6 month', 'TENURE_E 6-9 month', 'TENURE_F 9-12 month', 'TENURE_G 12-15 month',
                 'TENURE_H 15-18 month', 'TENURE_I 18-21 month', 'TENURE_J 21-24 month', 'TENURE_K > 24 month',
                 'REGION_DAKAR', 'REGION_DIOURBEL', 'REGION_FATICK', 'REGION_KAFFRINE', 'REGION_KAOLACK',
                 'REGION_KEDOUGOU', 'REGION_KOLDA', 'REGION_LOUGA', 'REGION_MATAM', 'REGION_OTHER',
                 'REGION_SAINT-LOUIS', 'REGION_SEDHIOU', 'REGION_TAMBACOUNDA', 'REGION_THIES', 'REGION_ZIGUINCHOR'])

    it1_df['MONTANT/FREQUENCE_RECH'] = it1_df['MONTANT'] / it1_df['FREQUENCE_RECH']
    it1_df['MONTANT/FREQUENCE_RECH'] = it1_df['MONTANT/FREQUENCE_RECH'].fillna(0)

    it1_df['REVENUE/FREQUENCE_RECH'] = it1_df['REVENUE'] / it1_df['FREQUENCE_RECH']
    it1_df['REVENUE/FREQUENCE_RECH'] = it1_df['REVENUE/FREQUENCE_RECH'].fillna(0)

    it1_df['DATA_VOLUME/REGULARITY'] = it1_df['DATA_VOLUME'] / it1_df['REGULARITY']
    it1_df['DATA_VOLUME/REGULARITY'] = it1_df['DATA_VOLUME/REGULARITY'].fillna(0)

    it1_df['ON_NET/REGULARITY'] = it1_df['ON_NET'] / it1_df['REGULARITY']
    it1_df['ON_NET/REGULARITY'] = it1_df['ON_NET/REGULARITY'].fillna(0)

    it1_df['ORANGE/REGULARITY'] = it1_df['ORANGE'] / it1_df['REGULARITY']
    it1_df['ORANGE/REGULARITY'] = it1_df['ORANGE/REGULARITY'].fillna(0)

    it1_df['TIGO/REGULARITY'] = it1_df['TIGO'] / it1_df['REGULARITY']
    it1_df['TIGO/REGULARITY'] = it1_df['TIGO/REGULARITY'].fillna(0)

    it1_df['ZONE1/REGULARITY'] = it1_df['ZONE1'] / it1_df['REGULARITY']
    it1_df['ZONE1/REGULARITY'] = it1_df['ZONE1/REGULARITY'].fillna(0)

    it1_df['ZONE2/REGULARITY'] = it1_df['ZONE2'] / it1_df['REGULARITY']
    it1_df['ZONE2/REGULARITY'] = it1_df['ZONE2/REGULARITY'].fillna(0)

    it1_df = convert_packs(it1_df)

    return it1_df



In [6]:
convert_df_to_it2(train_df, True).to_csv('iteration2.csv')

Результаты кластеризации:
Тренировочная выборка: 0.8616698789026064
Тестовая выборка: 0.8619286964480138


/var/folders/gp/7c7n5yqd14z8b8k4499t4r_m0000gn/T/ipykernel_936/3547515623.py:59: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  top_pack_y_df = top_pack_y_df.append(top_pack_predict_df['TOP_PACK']).sort_index()


Разнесено названий по категориям: [12, 26, 10, 23, 8, 6, 6, 8, 6, 4, 3, 3, 26]


In [11]:
convert_df_to_it2(test_df, False).to_csv('iteration2_test.csv')

File not found, training new model
Результаты кластеризации:
Тренировочная выборка: 0.8515773028140727
Тестовая выборка: 0.8523488948081825


/var/folders/gp/7c7n5yqd14z8b8k4499t4r_m0000gn/T/ipykernel_936/3172395905.py:61: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  top_pack_y_df = top_pack_y_df.append(top_pack_predict_df['TOP_PACK']).sort_index()


Разнесено названий по категориям: [19, 46, 20, 40, 14, 12, 10, 13, 9, 4, 6, 6, 45]


## Генератор набора данных "Третья итерация"


| № п/п | Акроним поля           | Принятое решение                                                   |
|-------|------------------------|--------------------------------------------------------------------|
| 1     | MRG                    | Удалить из набора данных                                           |
| 2     | REGION                 | Закодировать OneHot                                                |
| 3     | MONTANT                | Заполнить значением 0                                              |
| 4     | FREQUENCE_RECH         | Заполнить значением 0                                              |
| 5     | REVENUE                | Заполнить значением 0                                              |
| 6     | ARPU_SEGMENT           | Заполнить значением 0                                              |
| 7     | FREQUENCE              | Заполнить значением 0                                              |
| 8     | DATA_VOLUME            | Заполнить значением 0                                              |
| 9     | ON_NET                 | Заполнить значением 0                                              |
| 10    | ORANGE                 | Заполнить значением 0                                              |
| 11    | TIGO                   | Заполнить значением 0                                              |
| 12    | ZONE1                  | Заполнить значением 0                                              |
| 13    | ZONE2                  | Заполнить значением 0                                              |
| 14    | TENURE                 | Закодировать OneHot                                                |
| 15    | TOP_PACK               | Определенные значения, сведенные в группы закодировать OneHot      |
| 16    | MONTANT/FREQUENCE_RECH | Дополнительный параметр созданный делением соответствующих колонок |
| 17    | REVENUE/FREQUENCE_RECH | Дополнительный параметр созданный делением соответствующих колонок |
| 18    | DATA_VOLUME/REGULARITY | Дополнительный параметр созданный делением соответствующих колонок |
| 19    | ON_NET/REGULARITY      | Дополнительный параметр созданный делением соответствующих колонок |
| 20    | ORANGE/REGULARITY      | Дополнительный параметр созданный делением соответствующих колонок |
| 21    | TIGO/REGULARITY        | Дополнительный параметр созданный делением соответствующих колонок |
| 22    | ZONE1/REGULARITY       | Дополнительный параметр созданный делением соответствующих колонок |
| 23    | ZONE2/REGULARITY       | Дополнительный параметр созданный делением соответствующих колонок |


In [13]:
def convert_df_to_it3(it2_df: pd.DataFrame, is_train_df: bool) -> pd.DataFrame:
    it3_df = pd.get_dummies(it2_df, columns=['TENURE', 'REGION', 'TOP_PACK'])
    return it3_df

In [14]:
it2_df = pd.read_csv('iteration2.csv')
it2_df

,Unnamed: 0,user_id,MONTANT,FREQUENCE_RECH,REVENUE,ARPU_SEGMENT,FREQUENCE,DATA_VOLUME,ON_NET,ORANGE,...,REGION,TENURE,MONTANT/FREQUENCE_RECH,REVENUE/FREQUENCE_RECH,DATA_VOLUME/REGULARITY,ON_NET/REGULARITY,ORANGE/REGULARITY,TIGO/REGULARITY,ZONE1/REGULARITY,ZONE2/REGULARITY
0,0,00000bfd7d50f01092811bc0c8d7b0d6fe7c3596,4250.0,15.0,4251.0,1417.0,17.0,4.0,388.0,46.0,...,FATICK,K > 24 month,283.333333,283.400000,0.074074,7.185185,0.851852,0.018519,0.018519,0.037037
1,1,00000cb4a5d760de88fecb38e2f71b7bec52e834,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,OTHER,I 18-21 month,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,2,00001654a9d9f96303d9969d0a4a851714a4bb57,3600.0,2.0,1020.0,340.0,2.0,0.0,90.0,46.0,...,OTHER,K > 24 month,1800.000000,510.000000,0.000000,5.294118,2.705882,0.411765,0.000000,0.000000
3,3,00001dd6fa45f7ba044bd5d84937be464ce78ac2,13500.0,15.0,13502.0,4501.0,18.0,43804.0,41.0,102.0,...,DAKAR,K > 24 month,900.000000,900.133333,706.516129,0.661290,1.645161,0.032258,0.000000,0.000000
4,4,000028d9e13a595abe061f9b58f3d76ab907850f,1000.0,1.0,985.0,328.0,1.0,0.0,39.0,24.0,...,DAKAR,K > 24 month,1000.000000,985.000000,0.000000,3.545455,2.181818,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2154043,2154043,ffffe85215ddc71a84f95af0afb0deeea90e6967,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,OTHER,K > 24 month,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2154044,2154044,ffffeaaa9289cdba0ac000f0ab4b48f4aa74ed15,6100.0,15.0,5800.0,1933.0,15.0,621.0,26.0,40.0,...,THIES,K > 24 month,406.666667,386.666667,11.290909,0.472727,0.727273,0.727273,0.000000,0.000000
2154045,2154045,fffff172fda1b4bb38a95385951908bb92379809,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,OTHER,K > 24 month,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2154046,2154046,fffff5911296937a37f09a37a549da2e0dad6dbb,10000.0,11.0,7120.0,2373.0,13.0,0.0,0.0,140.0,...,THIES,K > 24 month,909.090909,647.272727,0.000000,0.000000,5.000000,0.464286,0.000000,0.000000


In [15]:
it3_df = convert_df_to_it3(it2_df, True)
it3_df

,Unnamed: 0,user_id,MONTANT,FREQUENCE_RECH,REVENUE,ARPU_SEGMENT,FREQUENCE,DATA_VOLUME,ON_NET,ORANGE,...,TOP_PACK_gprs,TOP_PACK_internat,TOP_PACK_mixt,TOP_PACK_month,TOP_PACK_net,TOP_PACK_other,TOP_PACK_pilot,TOP_PACK_unlimited,TOP_PACK_week,TOP_PACK_wifi
0,0,00000bfd7d50f01092811bc0c8d7b0d6fe7c3596,4250.0,15.0,4251.0,1417.0,17.0,4.0,388.0,46.0,...,0,0,0,0,0,0,0,1,0,0
1,1,00000cb4a5d760de88fecb38e2f71b7bec52e834,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,1,0,0
2,2,00001654a9d9f96303d9969d0a4a851714a4bb57,3600.0,2.0,1020.0,340.0,2.0,0.0,90.0,46.0,...,0,0,0,0,1,0,0,0,0,0
3,3,00001dd6fa45f7ba044bd5d84937be464ce78ac2,13500.0,15.0,13502.0,4501.0,18.0,43804.0,41.0,102.0,...,0,0,0,0,0,0,0,0,0,0
4,4,000028d9e13a595abe061f9b58f3d76ab907850f,1000.0,1.0,985.0,328.0,1.0,0.0,39.0,24.0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2154043,2154043,ffffe85215ddc71a84f95af0afb0deeea90e6967,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,1,0,0
2154044,2154044,ffffeaaa9289cdba0ac000f0ab4b48f4aa74ed15,6100.0,15.0,5800.0,1933.0,15.0,621.0,26.0,40.0,...,0,0,0,0,0,0,0,0,0,0
2154045,2154045,fffff172fda1b4bb38a95385951908bb92379809,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,1,0,0
2154046,2154046,fffff5911296937a37f09a37a549da2e0dad6dbb,10000.0,11.0,7120.0,2373.0,13.0,0.0,0.0,140.0,...,0,0,0,0,1,0,0,0,0,0


In [17]:
it3_df.to_csv('iteration3.csv')

In [18]:
it2_test_df = pd.read_csv('iteration2_test.csv')
it2_test_df

,Unnamed: 0,user_id,MONTANT,FREQUENCE_RECH,REVENUE,ARPU_SEGMENT,FREQUENCE,DATA_VOLUME,ON_NET,ORANGE,...,REGION,TENURE,MONTANT/FREQUENCE_RECH,REVENUE/FREQUENCE_RECH,DATA_VOLUME/REGULARITY,ON_NET/REGULARITY,ORANGE/REGULARITY,TIGO/REGULARITY,ZONE1/REGULARITY,ZONE2/REGULARITY
0,0,00001dbe00e56fc4b1c1b65dda63de2a5ece55f9,5000.0,5.0,5000.0,1667.0,5.0,0.0,378.0,11.0,...,THIES,K > 24 month,1000.000000,1000.000000,0.000000,9.000000,0.261905,0.119048,0.0,0.0
1,1,000055d41c8a62052dd426592e8a4a3342bf565d,300.0,2.0,326.0,109.0,3.0,397.0,0.0,0.0,...,OTHER,I 18-21 month,150.000000,163.000000,9.682927,0.000000,0.000000,0.000000,0.0,0.0
2,2,000081dd3245e6869a4a9c574c7050e7bb84c2c8,3300.0,25.0,3400.0,1133.0,26.0,7150.0,0.0,2.0,...,DAKAR,K > 24 month,132.000000,136.000000,125.438596,0.000000,0.035088,0.087719,0.0,0.0
3,3,0000b76d2145d9445d9ff6b65c9ebc4196c89337,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,OTHER,K > 24 month,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0
4,4,0000bae5480628cf8fe51ad84bcb39772fc79224,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,OTHER,K > 24 month,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
380122,380122,fffe7e03c7eede2ad0a728ee516c4d342dd16107,4000.0,8.0,3999.0,1333.0,8.0,1587.0,26.0,250.0,...,DAKAR,K > 24 month,500.000000,499.875000,29.943396,0.490566,4.716981,0.018868,0.0,0.0
380123,380123,fffec230e6a1aa51ab37d0051ece42de611e71c6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,OTHER,K > 24 month,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0
380124,380124,ffff0dcc1ab9812bf205b6d76e9d084053cd96f5,3950.0,7.0,3949.0,1316.0,10.0,1724.0,25.0,71.0,...,OTHER,K > 24 month,564.285714,564.142857,114.933333,1.666667,4.733333,0.000000,0.0,0.0
380125,380125,ffff91ea6a09a0c8ea42bc6ae33df4b5e06283dc,3850.0,18.0,3955.0,1318.0,23.0,2962.0,0.0,7.0,...,OTHER,K > 24 month,213.888889,219.722222,102.137931,0.000000,0.241379,0.000000,0.0,0.0


In [19]:
it3_test_df = convert_df_to_it3(it2_test_df, True)
it3_test_df

,Unnamed: 0,user_id,MONTANT,FREQUENCE_RECH,REVENUE,ARPU_SEGMENT,FREQUENCE,DATA_VOLUME,ON_NET,ORANGE,...,TOP_PACK_evc,TOP_PACK_internat,TOP_PACK_mixt,TOP_PACK_month,TOP_PACK_net,TOP_PACK_other,TOP_PACK_pilot,TOP_PACK_unlimited,TOP_PACK_week,TOP_PACK_wifi
0,0,00001dbe00e56fc4b1c1b65dda63de2a5ece55f9,5000.0,5.0,5000.0,1667.0,5.0,0.0,378.0,11.0,...,0,0,0,0,1,0,0,0,0,0
1,1,000055d41c8a62052dd426592e8a4a3342bf565d,300.0,2.0,326.0,109.0,3.0,397.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
2,2,000081dd3245e6869a4a9c574c7050e7bb84c2c8,3300.0,25.0,3400.0,1133.0,26.0,7150.0,0.0,2.0,...,0,0,0,0,0,0,0,0,0,0
3,3,0000b76d2145d9445d9ff6b65c9ebc4196c89337,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1,0,0,0,0,0,0,0,0,0
4,4,0000bae5480628cf8fe51ad84bcb39772fc79224,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
380122,380122,fffe7e03c7eede2ad0a728ee516c4d342dd16107,4000.0,8.0,3999.0,1333.0,8.0,1587.0,26.0,250.0,...,0,0,0,0,0,0,0,1,0,0
380123,380123,fffec230e6a1aa51ab37d0051ece42de611e71c6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1,0,0,0,0,0,0,0,0,0
380124,380124,ffff0dcc1ab9812bf205b6d76e9d084053cd96f5,3950.0,7.0,3949.0,1316.0,10.0,1724.0,25.0,71.0,...,0,0,0,0,0,0,0,0,0,0
380125,380125,ffff91ea6a09a0c8ea42bc6ae33df4b5e06283dc,3850.0,18.0,3955.0,1318.0,23.0,2962.0,0.0,7.0,...,0,0,0,0,0,0,0,0,0,0


In [20]:
it3_df.to_csv('iteration3_test.csv')